Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from itertools import product
from grader import Grader

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [3]:
# columns for which to create a unique key (date_block_num is the month number)
index_cols = ['date_block_num', 'shop_id', 'item_id']


In [4]:
# unique shops & items per month
shops = sales[['date_block_num', 'shop_id']].drop_duplicates()
items = sales[['date_block_num', 'item_id']].drop_duplicates()
 
# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = shops.loc[shops['date_block_num'] == block_num, 'shop_id']
    cur_items = items.loc[items['date_block_num'] == block_num, 'item_id']
    cur_items_shops = list(product(*[[block_num], cur_shops, cur_items]))
    grid += cur_items_shops


# convert to dataframe
grid = pd.DataFrame(np.array(grid), columns = index_cols, dtype=np.int32) 


In [5]:
# get aggregated sales for (shop_id, item_id, month)
monthly_sales = (sales
    .groupby(index_cols, as_index=False)
    .agg({'item_cnt_day':'sum'})
    .rename(columns={'item_cnt_day': 'target'})
)

# join aggregated data to the grid
all_data = pd.merge(grid, monthly_sales, how='left', on=index_cols).fillna(0)

# sort the data
all_data.sort_values(index_cols,inplace=True)
all_data.head()


,date_block_num,shop_id,item_id,target
139255,0,0,19,0.0
141495,0,0,27,0.0
144968,0,0,28,0.0
142661,0,0,29,0.0
138947,0,0,32,6.0


In [30]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
all_data.head()


/opt/conda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

In [56]:
'global mean: {:.4f}'.format(all_data['target'].mean())

'global mean: 0.3343'

#### Method 1

In [57]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])


0.483038698862


#### Method 2

In [ ]:
'''
Differently to `.target.mean()` function `transform` 
will return a dataframe with an index like in `all_data`.
Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [9]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [53]:
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


In [72]:
# YOUR CODE GOES HERE
k_fold = KFold(5, shuffle=False)
df_new = all_data.copy(deep=True)

for ftrain_idx, fval_idx in k_fold.split(all_data['target']):
    
    # split fold in train/val
    ftrain = df_new.iloc[ftrain_idx]
    means = ftrain.groupby('item_id')['target'].mean()
    df_new.loc[fval_idx, 'item_target_enc'] = df_new.loc[fval_idx, 'item_id'].map(means)

# fill missing values with the global target mean
df_new['item_target_enc'].fillna(0.3343, inplace=True) 


In [73]:
# encoded feature
encoded_feature = df_new['item_target_enc']

# correlation coefficient
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

#0.409235596684
#0.483038698862

0.41650796922
Current answer for task KFold_scheme is: 0.41650796922


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [ ]:
# for each row i, the mean encoding is:
# ° the sum of its class minus its value, 
# ° divided by the number of items of the class minus one


In [20]:
# YOUR CODE GOES HERE
df_new = all_data.copy(deep=True)
df_new['item_target_enc'] = (
    (df_new.groupby('item_id')['target'].transform('sum') - df_new['target']) / # sum minus own value
    (df_new.groupby('item_id')['target'].transform('count') - 1)                # count minus one
).fillna(0.3343)

df_new.head()


,date_block_num,shop_id,item_id,target,item_target_enc
139255,0,0,19,0.0,0.022727
141495,0,0,27,0.0,0.056911
144968,0,0,28,0.0,0.141414
142661,0,0,29,0.0,0.037500
138947,0,0,32,6.0,1.316088


In [21]:
# encoded feature
encoded_feature = df_new['item_target_enc']

# correlation coefficient
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)


0.480384831129
Current answer for task Leave-one-out_scheme is: 0.480384831129


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [22]:
# YOUR CODE GOES HERE
df_new = all_data.copy(deep=True)
df_new['item_target_enc'] = (
    (
        (df_new.groupby('item_id')['target'].transform('mean') *
         df_new.groupby('item_id')['target'].transform('count'))
        + 0.3343 * 100
    ) /
    (df_new.groupby('item_id')['target'].transform('count') + 100)
)


In [23]:
# encoded feature
encoded_feature = df_new['item_target_enc']

# correlation coefficient
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)


0.48181987971
Current answer for task Smoothing_scheme is: 0.48181987971


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [6]:
# YOUR CODE GOES HERE
cumsum = all_data.groupby('item_id')['target'].cumsum() - all_data['target'] # cumsum of [0..n-1]
cumcnt = all_data.groupby('item_id')['target'].cumcount()                    # cumcnt of [0..n-1]


In [16]:
encoded_feature = (cumsum / cumcnt).fillna(0.3343)


In [17]:
# correlation coefficient
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)


0.502524521108
Current answer for task Expanding_mean_scheme is: 0.502524521108


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [76]:
STUDENT_EMAIL = 'plat.sebastien@hotmail.fr'
STUDENT_TOKEN = 'ci9hrLTZRKeJe3yZ'
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.41650796922
Task Leave-one-out_scheme: 0.480384831129
Task Smoothing_scheme: 0.48181987971
Task Expanding_mean_scheme: 0.502524521108


In [77]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
